In [ ]:
!pip install faiss-cpu --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 57.2 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-openai langchain-faiss --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
# !pip install huggingface_hub
!pip install langchain_community --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install langchain-huggingface --q

In [ ]:
!pip install sentence-transformers --q

In [ ]:
from sentence_transformers import CrossEncoder
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
import os, getpass

In [ ]:
os.environ['HF_TOKEN'] = getpass.getpass()

··········


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
vectorstore = FAISS.from_texts(
    [
        "Cross encoders score query-document pairs",
        "FAISS is a fast vector search library",
        "LangChain changed retriever imports after 1.0",
        "Compression is different from reranking"
    ],
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
retriever.invoke("How does cross encoding work?")

[Document(id='b27ff8e0-9ac0-469f-9e5e-3c4a181182f9', metadata={}, page_content='Cross encoders score query-document pairs'),
 Document(id='6d4f31c8-2670-43ee-b247-0ed9d5ece879', metadata={}, page_content='Compression is different from reranking'),
 Document(id='35a1f722-3d1f-49fe-a569-182c4e8839f9', metadata={}, page_content='FAISS is a fast vector search library'),
 Document(id='ee2af75d-0864-4ba8-bc79-a85699922def', metadata={}, page_content='LangChain changed retriever imports after 1.0')]

In [ ]:
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
def retrieve_and_rerank(query, top_n=5):
    docs = retriever.invoke(query)

    pairs = [(query, d.page_content) for d in docs]
    scores = reranker.predict(pairs)

    reranked = sorted(
        zip(scores, docs),
        key=lambda x: x[0],
        reverse=True
    )

    return [doc for _, doc in reranked[:top_n]]

In [ ]:
docs = retrieve_and_rerank("How does cross encoding work?")

In [ ]:
for d in docs:
    print(d.page_content)

Cross encoders score query-document pairs
FAISS is a fast vector search library
Compression is different from reranking
LangChain changed retriever imports after 1.0
